In [18]:
import os
import openai
from langchain.embeddings import OpenAIEmbeddings
from langchain.vectorstores import AzureSearch
from dotenv import load_dotenv
load_dotenv('.env')
print(os.getenv('SEARCH_SERVICE_NAME'), os.getenv('SEARCH_API_KEY'), os.getenv('SEARCH_INDEX_NAME'))



https://servicio-busqueda.search.windows.net pxK15ux30XO9deLtPU1jggrivkk6Ht2bUt9ukCWno9AzSeAFhvK7 indice-rag


In [ ]:
embeddings = OpenAIEmbeddings(deployment="text-embedding-ada-002", chunk_size=1)

# Connect to Azure Cognitive Search
acs = AzureSearch(azure_search_endpoint=os.getenv('SEARCH_SERVICE_NAME'),
                 azure_search_key=os.getenv('SEARCH_API_KEY'),
                 index_name=os.getenv('SEARCH_INDEX_NAME'),
                 embedding_function=embeddings.embed_query)

In [20]:
from langchain.document_loaders import CSVLoader

loader = CSVLoader("wine-ratings.csv")
print(loader)
documents = loader.load()

In [21]:
from langchain.text_splitter import CharacterTextSplitter
text_splitter = CharacterTextSplitter(chunk_size=1000, chunk_overlap=0)
print(documents[0])
docs = text_splitter.split_documents(documents)
docs = docs[:100]
print(docs[0])
acs.add_documents(documents=docs)

page_content=': 0\nname: 1000 Stories Bourbon Barrel Aged Batch Blue Carignan 2016\ngrape: \nregion: Mendocino, California\nvariety: Red Wine\nrating: 91.0\nnotes: This is a very special, limited release of 1000 Stories Bourbon Barrel-Aged Carignan, their first-ever release of Carignan as a single varietal. Classic and rustic with a little edge. Look for notes of brilliantly racy red and black fruits set to a rich backdrop of toast, herbs, and cocoa.' metadata={'source': 'wine-ratings.csv', 'row': 0}
page_content=': 0\nname: 1000 Stories Bourbon Barrel Aged Batch Blue Carignan 2016\ngrape: \nregion: Mendocino, California\nvariety: Red Wine\nrating: 91.0\nnotes: This is a very special, limited release of 1000 Stories Bourbon Barrel-Aged Carignan, their first-ever release of Carignan as a single varietal. Classic and rustic with a little edge. Look for notes of brilliantly racy red and black fruits set to a rich backdrop of toast, herbs, and cocoa.' metadata={'source': 'wine-ratings.csv'

['OWVlOWI0ZjItMmZmOC00YzdlLTgzMzQtNjE2NmQ3NzQ0M2Q0',
 'NjlhNmE2YjMtZjVlMC00YWFmLTg1OTMtMGU0NDI3NDJkMTY2',
 'NTk1ZTM3ZmItN2IzNS00NzNhLWFlNGUtYTQzNDZlOGQzZjdl',
 'NDBjY2JjNjctMzM4Mi00NTU5LTlhODItMTFjODFjYTkyNTk1',
 'ZjkxM2U4YTktZGI4NS00ZjIxLWExM2UtNDIxYzY5ODU1M2I1',
 'ZmZjMDk3ODYtMDY4ZC00YjU3LWI2MWYtNzZkYWU0NmJlMmFj',
 'MWY0NTZjYWEtN2FlZS00NjA5LWJhOWQtY2RhMWIxZmE5YjNi',
 'MmE2Njk3MGMtZjUyMS00NjI4LTgxODMtOWY1OGQ0NDU5ODRi',
 'OWYyZDM5YzctYzYzMi00MzlhLTk1NWMtYjdkMTA1NWQ0MmZh',
 'OTYzNzg4MDctZjgzZC00NTgzLWJiMDctZTkyMGU2ZGM0Zjhl',
 'MDZjNmI0MzItZjQ4MS00NDQ5LTg0OWEtMzk3NzVhMzUxOWMy',
 'MjI2OWI4ZjktOWY5OC00MGI1LTkxMDMtOGUzMjYyNTEzYTY3',
 'MjA1NGNjYmItMmQ1My00M2JjLTg2ZTAtYzQzMWEyYTAzNWRh',
 'N2FmZDI4ODItNzc0My00NjllLWEwZjEtMDA0ZDUyNGNkOGNh',
 'NDU1YjllMTEtOWFmOC00NzFiLWIzNDctZmJhZjViZmI3YTAz',
 'ODBmMDYwYTItYmVhMC00YWJjLTlhMmMtMTNmOTQzNjJlNDI2',
 'MTg5MDY2NmEtMGE2My00MDgwLWFlYmItMjkwNjcyNmNiYzg0',
 'ZDhkZGM3MWMtNTJhYS00YjEyLWJiZTAtOTk0YjBmMWZjMWU2',
 'NTkyOGM3ZjEtMTk3MS00NzJkLWE0MzctODBjZDYzNzRl

In [22]:
docs = acs.similarity_search_with_relevance_scores(
    query="What is the best Cabernet Sauvignon wine in Napa Valley above 94 points",
    k=5,
)
print(docs[0][0].page_content)
print(dir(docs[0][0]))

: 20
name: 1849 Declaration Napa Valley Cabernet Sauvignon 2014
grape: 
region: Napa Valley, California
variety: Red Wine
rating: 91.0
notes: The palate is robust with flavors of dark blueberry, blackberry, traces of red currant, and subtle sweet oak from the barrel. This wine is fruit forward, full-bodied and spreads richly across the palate with soft velvety tannins and a long-lasting finish.
['Config', '__abstractmethods__', '__annotations__', '__class__', '__class_vars__', '__config__', '__custom_root_type__', '__delattr__', '__dict__', '__dir__', '__doc__', '__eq__', '__exclude_fields__', '__fields__', '__fields_set__', '__format__', '__ge__', '__get_validators__', '__getattribute__', '__getstate__', '__gt__', '__hash__', '__include_fields__', '__init__', '__init_subclass__', '__iter__', '__json_encoder__', '__le__', '__lt__', '__module__', '__ne__', '__new__', '__post_root_validators__', '__pre_root_validators__', '__pretty__', '__private_attributes__', '__reduce__', '__reduce_ex

In [24]:
openai.api_base = os.getenv("OPENAI_API_BASE")  # Your Azure OpenAI resource's endpoint value.
openai.api_key = os.getenv("OPENAI_API_KEY")
openai.api_type = "azure"
openai.api_version = "2023-05-15" 
messages=[
    {"role": "system", "content": "Asisstant is a chatbot that helps you find the best wine for your taste."},
    {"role": "user", "content": "What is the best wine in Oregon above 92 points?"},
    {"role": "assistant", "content": docs[0][0].page_content}
]

response = openai.ChatCompletion.create(
    engine="gpt-4o-mini",
    messages=messages,
)
from pprint import pprint
pprint(response)
print(response['choices'][0]['message']['content'])

{'choices': [{'finish_reason': 'stop',
              'index': 0,
              'message': {'annotations': [],
                          'content': 'Oregon is renowned for its Pinot Noir, '
                                     'especially from regions like Willamette '
                                     "Valley. While I can't access real-time "
                                     'data to provide the latest specific '
                                     'wines above 92 points, some of the '
                                     'consistently highly-rated Oregon wines '
                                     'include:\n'
                                     '\n'
                                     '1. **Domaine Serene** - Known for their '
                                     'Pinots and Chardonnays, several vintages '
                                     'have received scores above 92.\n'
                                     '2. **Ken Wright Cellars** - Their Pinot '
                 